In [2]:
import pandas as pd
import numpy as np
from surprise import Dataset, Reader, SVD, KNNBasic, KNNWithMeans, KNNWithZScore, NMF
from surprise.model_selection import train_test_split
from surprise.accuracy import rmse
from collections import defaultdict

In [3]:
# 1. Загрузка и разделение данных
def load_and_split_data(file_path, test_size=0.3, random_state=42):
    
    # Загрузка данных
    df = pd.read_csv(file_path)
    reader = Reader(rating_scale=(0.5, 5))
    data = Dataset.load_from_df(df[['userId', 'movieId', 'rating']], reader)
    
    # Разделение данных
    trainset, testset = train_test_split(data, test_size=test_size, random_state=random_state)
    return trainset, testset

# 2. User-based коллаборативная фильтрация
def user_based_cf(trainset, testset):
    sim_options = {'name': 'cosine', 'user_based': True}
    algo = KNNWithMeans(sim_options=sim_options, verbose=False)
    algo.fit(trainset) 
    predictions = algo.test(testset)
    return rmse(predictions)

# 3. Item-based коллаборативная фильтрация
def item_based_cf(trainset, testset):
    sim_options = {'name': 'cosine', 'user_based': False}
    algo = KNNWithMeans(sim_options=sim_options, verbose=False)
    algo.fit(trainset)
    predictions = algo.test(testset)
    return rmse(predictions)

# 4. Рекомендательная система на основе SVD (аналог кластеризации)
def svd_based_cf(trainset, testset):
    algo = SVD(n_factors=100, random_state=42)
    algo.fit(trainset)
    predictions = algo.test(testset)
    return rmse(predictions)

# 5.1 Рекомендательная система на основе NMF (аналог кластеризации с учетом z-оценки)
def nmf_based_cf(trainset, testset):
    algo = NMF(n_factors=50, random_state=42)
    algo.fit(trainset)
    predictions = algo.test(testset)
    return rmse(predictions)

# 5.2 Рекомендательная система на основе кластеризации с учетом z-оценки
def knnz_based_cf(trainset, testset):
    sim_options = {'name': 'cosine', 'user_based': False}
    algo = KNNWithZScore(sim_options=sim_options, verbose=False)
    algo.fit(trainset)
    predictions = algo.test(testset)
    return rmse(predictions)

# 6. Рекомендательная система на основе голосования (усредненное предсказание)
def voting_based_cf(trainset, testset):
    # Используем несколько алгоритмов для голосования
    algos = [
        KNNWithMeans(sim_options={'name': 'cosine', 'user_based': True}, verbose=False),
        KNNWithMeans(sim_options={'name': 'cosine', 'user_based': False}, verbose=False),
        SVD(n_factors=100, random_state=42),
        NMF(n_factors=50, random_state=42)
    ]
    
    # Обучаем все алгоритмы
    for algo in algos:
        algo.fit(trainset)
    
    # Предсказываем рейтинги и усредняем их
    predictions = defaultdict(list)
    for algo in algos:
        for prediction in algo.test(testset):
            uid, iid, _, est, _ = prediction
            predictions[(uid, iid)].append(est)
    
    # Усредняем предсказания
    final_predictions = []
    for (uid, iid), preds in predictions.items():
        true_r = next((true_r for (u, i, true_r) in testset if u == uid and i == iid), None)
        if true_r is not None:
            final_predictions.append((uid, iid, true_r, np.mean(preds), {'n_votes': len(preds)}))
    
    return rmse(final_predictions)

# Оформление результа
def create_results_dataframe(results):
    """
    Создает pandas DataFrame из словаря результатов.
    
    :param results: Словарь с названиями методов в качестве ключей и RMSE в качестве значений
    :return: pandas DataFrame с результатами
    """
    df = pd.DataFrame(list(results.items()), columns=['Метод', 'RMSE'])
    df.set_index('Метод', inplace=True)
    return df

# Основная функция
def main():
    # Загрузка и разделение данных
    trainset, testset = load_and_split_data('data/ml-latest-small/ratings.csv', test_size=0.05)
    
    # Словарь для хранения результатов
    results = {}
    
    # User-based CF
    results['User-based CF'] = user_based_cf(trainset, testset)
    
    # Item-based CF
    results['Item-based CF'] = item_based_cf(trainset, testset)
    
    # SVD-based CF (аналог кластеризации)
    results['SVD-based CF'] = svd_based_cf(trainset, testset)
    
    # NMF-based CF (аналог кластеризации с учетом z-оценки)
    results['NMF-based CF'] = nmf_based_cf(trainset, testset)

    #KNN-Z CF
    results['KNN-Z CF'] = knnz_based_cf(trainset, testset)
    
    # Voting-based CF
    results['Voting-based CF'] = voting_based_cf(trainset, testset)
    
    # Создаем DataFrame с результатами
    results_df = create_results_dataframe(results)
    
    # Выводим результаты
    print(results_df.sort_values('RMSE'))
    
    # results_df.to_csv('recommender_systems_results.csv')
# if __name__ == "__main__":
#     main()